# Interaction between two classes (0 and 1) in the COIL 20 dataset

For this, we will use the induced block function.

In [ ]:
from PIL import Image
import numpy as np
from numpy.random import default_rng
import os
from skimage.metrics import structural_similarity as ssim

import IBloFunMatch_inter as ibfm

import matplotlib.pyplot as plt
import networkx as nx
from scipy.spatial import distance

In [ ]:
def draw_repr_cycle(repr_cycle, figsize, data):
    fig, ax = plt.subplots(figsize=figsize)
    G = nx.Graph()
    for v in np.unique(repr_cycle):
        G.add_node(v)
    weighted_edges = []
    for edge in np.array(repr_cycle).reshape((-1,2)).tolist():
        weighted_edges.append((edge[0], edge[1], distance.euclidean(data[edge[0]], data[edge[1]])))
    # G.add_edges_from(weighted_edges)
    G.add_weighted_edges_from(weighted_edges)
    pos = nx.spectral_layout(G)
    nx.draw_networkx(G, ax=ax, pos=pos, width=figsize[0])
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    for node in pos.keys():
        im_array = data[node].reshape((128,128))
        im_frame = Image.fromarray(im_array)
        center = origin=pos[node]
        extent = (center[0]-0.1, center[0]+0.1, center[1]-0.1, center[1]+0.1)
        ax.imshow(im_array, cmap="gray", extent=extent, zorder=4)
    
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
    return ax, pos

In [ ]:
CLASS_LIST = [0,4,5]
NUM_SAMPLES = 72

Read both classes 0 and 1 and store into `data` array

In [ ]:
# Create list of labels
y=[]
for c in CLASS_LIST:
    y += [c]*NUM_SAMPLES
y = np.array(y)
# Read data
data = []
for c in CLASS_LIST:
    for i in range(NUM_SAMPLES):
        im_frame = Image.open(f"data_COIL20/coil-20-proc/obj{c+1}__{i}.png")
        np_frame = np.array(im_frame)
        data.append(np_frame.ravel())
    # samples per class
# going through classes
# Store data into variable
data = np.array(data)
print("All data shape")
print(data.shape)

Compute matrix of structural symilarity distances.

In [ ]:
Dist_X = np.zeros((data.shape[0],data.shape[0]))
for i in range(data.shape[0]):
    for j in range(data.shape[0]):
        if i < j:
            Dist_X[i][j] = -np.log(ssim(data[i], data[j]))
        elif i > j:
            Dist_X[i][j] = Dist_X[j][i]
    # end for over columns 
# for over rows

Compute block function from both classes to dataset.

In [ ]:
exp_ibfm = []
output_dir = "output"
for cidx in CLASS_LIST:
    class_indices = np.nonzero(np.array(y)==cidx)[0].tolist()
    Dist_S = Dist_X[class_indices][:,class_indices]
    exp_ibfm.append(ibfm.get_IBloFunMatch_subset(Dist_S, Dist_X, class_indices, output_dir, num_it=4, max_rad=-1))
# end for over classes 

In [ ]:
os.makedirs(f"plots/COIL_CYCLES_MIX", exist_ok=True)
X_barcode_1 = exp_ibfm[0]["X_barcode_1"]
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5,3))
ibfm.plot_barcode(X_barcode_1, "navy", ax)
plt.savefig(f"plots/COIL_CYCLES_MIX/total_barcode.png")

In [ ]:
long_X_bars = ((exp_ibfm[0]["X_barcode_1"][:,1]-exp_ibfm[0]["X_barcode_1"][:,0])>0.15).tolist()
long_X_bars = np.nonzero(long_X_bars)[0].tolist()
print(exp_ibfm[0]["X_barcode_1"][long_X_bars])

In [ ]:
def plot_cycle_components(repr_cycle, data):
    G = nx.Graph()
    for v in np.unique(repr_cycle):
        G.add_node(v)
    weighted_edges = []
    for edge in np.array(repr_cycle).reshape((-1,2)).tolist():
        weighted_edges.append((edge[0], edge[1], distance.euclidean(data[edge[0]], data[edge[1]])))
    G.add_weighted_edges_from(weighted_edges)
    S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

    figsize=(8*len(S), 8)
    fig, ax = plt.subplots(ncols=len(S), nrows=1, figsize=figsize)
    if len(S)==1:
        ax = [ax]
    for i, sub in enumerate(S):
        pos = nx.spectral_layout(sub)
        nx.draw_networkx(sub, ax=ax[i], pos=pos, width=figsize[0])
        xlim = ax[i].get_xlim()
        ylim = ax[i].get_ylim()
        for node in pos.keys():
            im_array = data[node].reshape((128,128))
            im_frame = Image.fromarray(im_array)
            center = origin=pos[node]
            extent = (center[0]-0.1, center[0]+0.1, center[1]-0.1, center[1]+0.1)
            ax[i].imshow(im_array, cmap="gray", extent=extent, zorder=4)
        
        ax[i].set_ylim(ylim)
        ax[i].set_xlim(xlim)
    # end for 
    return ax


In [ ]:
# %%capture
for id, repr_cycle in enumerate(exp_ibfm[0]["X_reps_1"]):
    if id not in long_X_bars:
        continue
    print(f"Cycle {id}, number of elements: {len(np.unique(repr_cycle))}")
    print(np.sort(np.unique(repr_cycle)))
    figsize = (len(repr_cycle), len(repr_cycle))
    figsize = (8,12)
    ax = plot_cycle_components(repr_cycle, data)
    os.makedirs(f"plots/COIL_CYCLES_MIX/codomain", exist_ok=True)
    plt.savefig(f"plots/COIL_CYCLES_MIX/codomain/cycle_rep_codomain_{id}.png")

Get the indices of the intervals which are not in the image of any of the block functions from classes to the total.

In [ ]:
indices_image_block_1 = [] 
for ibfm_out in exp_ibfm:
    indices_image_block_1 += np.unique(ibfm_out["block_function_1"]).tolist()

not_indices_block_1 = [i for i in range(exp_ibfm[0]["X_barcode_1"].shape[0]) if i not in indices_image_block_1]

In [ ]:
for ibfm_out in exp_ibfm:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
    ibfm.plot_matching(ibfm_out, ax, fig, block_function=True, codomain_int=not_indices_block_1)

Now, repeat the same experiment for dimension 0

In [ ]:
indices_image_block_0 = [] 
for ibfm_out in exp_ibfm:
    indices_image_block_0 += np.unique(ibfm_out["block_function_0"]).tolist()
not_indices_block_0 = [i for i in range(exp_ibfm[0]["X_barcode_0"].shape[0]) if i not in indices_image_block_0]

In [ ]:
for ibfm_out in exp_ibfm:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
    ibfm.plot_matching(ibfm_out, ax, fig, block_function=True, codomain_int=not_indices_block_0, dim=0)

In [ ]:
not_indices_block_0